In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras.layers import TextVectorization
from tensorflow import data
from keras import layers, models

In [ ]:
df = pd.read_csv('../../../../mnt/c/NN/NLP/IMDB Dataset.csv')
reviews = df['review']
sentiment = df['sentiment']

In [ ]:
def func(data):
    if (data == 'positive'):
        return 1
    else:
        return 0

In [ ]:
sentiment = sentiment.apply(func)

In [ ]:
max_features = 40000
vectorizer = TextVectorization(max_tokens=max_features, output_sequence_length=4000, output_mode='int')
vectorizer.adapt(reviews.values)

In [ ]:
vectorized_text = vectorizer(reviews.values)

In [ ]:
vectorized_text = np.array(vectorized_text)
sentiment = np.array(sentiment)

In [ ]:
dataset = data.Dataset.from_tensor_slices((vectorized_text, sentiment))
dataset = dataset.cache()
dataset = dataset.shuffle(50000)
dataset = dataset.batch(64)
dataset = dataset.prefetch(32)

reviews, sentiments = dataset.as_numpy_iterator().next()

In [ ]:
train_size = int(len(dataset) * 0.7)
val_size = int(len(dataset) * 0.2)
test_size = int(len(dataset) * 0.1) + 1

In [ ]:
train = dataset.take(train_size)
val = dataset.skip(train_size).take(val_size)
test = dataset.skip(train_size + val_size).take(test_size)

In [ ]:
train_generator=train.as_numpy_iterator()
batch=train_generator.next()

In [ ]:
randomize = np.arange(np.size(reviews))
np.random.shuffle(randomize)
training_data = vectorized_text[randomize]
training_labels = sentiment[randomize]

test_data = training_data[int(0.8 * np.size(training_labels)):, :]
test_labels = training_labels[int(0.8 * np.size(training_labels)):]

training_data = training_data[:int(0.8 * np.size(training_labels)), :]
training_labels = training_labels[:int(0.8 * np.size(training_labels))]

In [ ]:
del(reviews)
del(sentiment)
del(randomize)
del(vectorized_text)

In [ ]:
model = models.Sequential()

model.add(layers.Embedding(max_features + 1, 32, input_length=4000))

model.add(layers.Bidirectional(layers.LSTM(64, activation='tanh')))

model.add(layers.Dense(256, activation='tanh'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(128, activation='tanh'))

model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(loss=keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam())

In [ ]:
history=model.fit(train, epochs=16, validation_data=val)